In [ ]:
# default_exp simulation

# Simulation

> Tools for  simulating cellular vertex dynamics. Iterative algorithm implementations, cell monolayer generators (vertices). (anything else to add?!)

[a little bit more detail here]

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from vertex_simulation.primitives import *
import torch, numpy as np, matplotlib.pylab as plt
# from mpl_toolkits.mplot3d import axes3d
# after you modify this notebook use "nbdev_build_docs" to build the docs (and readme.md)
# and nbdev_clean_nbs to remove notebook metadata before pushing it
%load_ext autoreload
%autoreload 2